In [ ]:
#pip install pyprojroot

In [ ]:
#pip install chromadb

In [ ]:
#pip install openai

In [ ]:
from pyprojroot import here
import pandas as pd
import chromadb
from openai import OpenAI

<b>Create LLM client</b>

In [ ]:
client = OpenAI(api_key="")

<b>Create vector DB client</b>

In [ ]:
chroma_client = chromadb.PersistentClient(path=str(here("/Users/ashokarulsamy/Python Learning/itam/data/chroma_db/chroma_itam_ada")))

<b>Create collection in vector DB</b>

In [ ]:
#Already executed
collection = chroma_client.create_collection(name="itam_hw")

<b>Load data from csv file into data frame</b>

In [ ]:
file_dir = here("/Users/ashokarulsamy/Python Learning/itam/data/itam_files/itam_hw.csv")
df = pd.read_csv(file_dir, nrows=100)

In [ ]:
df

<b>Create chunk and generate vector represenattion for chunk using embedding model</b>

In [ ]:
#Already executed
docs = []
metadatas = []
ids = []
embeddings = []
for index, row in df.iterrows():
    output_str = ""
    # Treat each row as a separate chunk
    for col in df.columns:
        output_str += f"{col}: {row[col]},\n"
    response = client.embeddings.create(input = [output_str], model="text-embedding-ada-002").data[0].embedding
    embeddings.append(response)
    docs.append(output_str)
    metadatas.append({"source": "itam_hw"})
    ids.append(f"id{index}")

In [ ]:
docs

In [ ]:
print(metadatas)
print(ids)

In [ ]:
embeddings[0][:10]

<b>Vector embedding - Load vector represenation in vector DB</b>

In [ ]:
#Already executed
collection.add(
    documents=docs,
    metadatas=metadatas,
    embeddings=embeddings,
    ids=ids
)

In [ ]:
print("Number of vectors in vectordb:", collection.count())

<b>Vector embedding for user query</b>

In [ ]:
user_query = "I'm Ashok Arulsamy, list out assets assigned to me"
#user_query = "List of retired assets"
#user_query = "list of assets for which warrenty expires in the year 2025"
#user_query = "How many hardware assets are there" # try agent as semantic search will return huge volume of data
user_query_embeddings = client.embeddings.create(input = [user_query], model="text-embedding-ada-002").data[0].embedding

In [ ]:
user_query_embeddings

<b>Create vector DB instance</b>

In [ ]:
vectordb = chroma_client.get_collection(name="itam_hw")
vectordb.count()

<b>Semantic search for user query against vector DB</b>

In [ ]:
retrieved_context = vectordb.query(
    query_embeddings = user_query_embeddings,
    n_results=5 #top_k
)

retrieved_context

<b>Generate user friendly message using LLM</b>

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant that provides detailed answers."},
        {"role": "user", "content": f"User's question: {user_query}"},
        {"role": "assistant", "content": f"Retrieved content: {retrieved_context}"}
    ],
    temperature=0.0,  # Controls creativity (0.0 = deterministic, 1.0 = creative)
    max_tokens=1000,   # Adjust as needed
)

In [ ]:
print("Generated Answer:")
print(response.choices[0].message.content)